In [1]:
from IPython.display import display
from typing import List,Dict
import os
import sys
os.chdir('/home/rchuzh/programming/image_labelling_shrdc')
from pathlib import Path
Path.cwd()
SRC = Path('/home/rchuzh/programming/image_labelling_shrdc/src')  # ROOT folder -> ./src
LIB_PATH = SRC / "lib"

if str(LIB_PATH) not in sys.path:
    sys.path.insert(0, str(LIB_PATH))  # ./lib
else:
    pass

In [2]:
dsn = "host=localhost port=5432 dbname=eye user=shrdc password=shrdc"
def init_connection(dsn=None, connection_factory=None, cursor_factory=None, **kwargs):
    """ Connect to the PostgreSQL database server """

    try:
        # read connection parameters
        # params = config()

        # connect to the PostgreSQL server
        display('Connecting to the PostgreSQL database...')
        if kwargs:
            conn = psycopg2.connect(**kwargs)

        else:
            conn = psycopg2.connect(dsn, connection_factory, cursor_factory)

        # create a cursor
        with conn:
            with conn.cursor() as cur:

                # execute a statement
                cur.execute('SELECT version();')
                conn.commit()

                # display the PostgreSQL database server version
                db_version = cur.fetchall()[0]
                display(f"PostgreSQL database version: {db_version}")
                display(f"PostgreSQL connection status: {conn.info.status}")
                display(
                    f"You are connected to database '{conn.info.dbname}' as user '{conn.info.user}' on host '{conn.info.host}' at port '{conn.info.port}'.")
        return conn
    except (Exception, psycopg2.DatabaseError) as error:
        display(error)
        conn = None

In [3]:
import psycopg2

In [4]:
conn=init_connection(dsn)

'Connecting to the PostgreSQL database...'

"PostgreSQL database version: ('PostgreSQL 13.3 (Ubuntu 13.3-1.pgdg20.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.3.0-17ubuntu1~20.04) 9.3.0, 64-bit',)"

'PostgreSQL connection status: 0'

"You are connected to database 'eye' as user 'shrdc' on host 'localhost' at port '5432'."

In [5]:
from typing import List
def check_if_exist( context: List,conn):
        check_exist_SQL = """
                            SELECT
                                EXISTS (
                                    SELECT
                                        %s
                                    FROM
                                        public.dataset
                                    WHERE
                                        name = %s);
                        """
        exist_status = db_fetchone(check_exist_SQL, context,conn)[0]
        return exist_status

In [6]:
context=['name','asdfads']

In [5]:
def db_fetchone(sql_message, conn,vars: List=None):
    with conn:
        with conn.cursor() as cur:
            try:
                if vars:
                    cur.execute(sql_message, vars)
                else:
                    cur.execute(sql_message)
                conn.commit()
                return_one = cur.fetchone()  # return tuple
                return return_one
            except psycopg2.Error as e:
                display(e)

In [8]:
db_fetchone('select * from public.project',conn)

In [10]:
db_fetchone("select exists(select name from public.project where name = 'hello')",conn)

(False,)

In [11]:
context

['name', 'asdfads']

In [13]:
db_fetchone("select exists(select %s from public.project where name = %s)",conn,context)

(False,)

In [6]:
def db_fetchall(sql_message, conn,vars: List=None):
    with conn:
        with conn.cursor() as cur:
            try:
                if vars:
                    cur.execute(sql_message, vars)
                else:
                    cur.execute(sql_message)
                conn.commit()
                return_all = cur.fetchall()  # return tuple
                return return_all
            except psycopg2.Error as e:
                display(e)

In [21]:
query_dataset_SQL="""SELECT
    id,
    name,
    dataset_size,
    updated_at
FROM
    public.dataset;"""

dataset_list=db_fetchall(query_dataset_SQL,conn)
display(dataset_list)

[(1,
  'My First Dataset',
  None,
  datetime.datetime(2021, 7, 20, 11, 56, 43, 781535, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=480, name=None))),
 (2,
  'My Second Dataset',
  4,
  datetime.datetime(2021, 7, 20, 14, 35, 18, 643911, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=480, name=None))),
 (3,
  'My Third Dataset',
  4,
  datetime.datetime(2021, 7, 20, 14, 38, 58, 453171, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=480, name=None)))]

In [125]:
updated_at=[]
dataset_name=[]
for dataset in dataset_list:
    without_timezone=dataset[3].strftime('%Y-%m-%d %H:%M:%S')
    updated_at.append(without_timezone)
    dataset_name.append(dataset[1])

display(dataset_name,updated_at)
    

['My First Dataset', 'My Second Dataset', 'My Third Dataset']

['2021-07-20 11:56:43', '2021-07-20 14:35:18', '2021-07-20 14:38:58']

In [23]:
from datetime import datetime

In [118]:
str(updated_at[0])

'2021-07-20 11:56:43.781535+08:00'

In [83]:
updated_at[0].tzinfo

psycopg2.tz.FixedOffsetTimezone(offset=480, name=None)

In [119]:
updated_at[0].strftime("%d-%m-%Y %H:%M:%S")

'20-07-2021 11:56:43'

In [120]:
(updated_at[0].strftime('%Y-%m-%d %H:%M:%S'))

'2021-07-20 11:56:43'

In [8]:
import pandas as pd

In [130]:
df = pd.DataFrame( {'Dataset Name':dataset_name ,
    'Date':updated_at})
df

,Dataset Name,Date
0,My First Dataset,2021-07-20 11:56:43
1,My Second Dataset,2021-07-20 14:35:18
2,My Third Dataset,2021-07-20 14:38:58


In [ ]:
df['Date']=df['Date'].dt.tz_localize(None)


In [132]:

df['Date']=pd.to_datetime(df['Date'],format='%Y-%m-%d %H:%M:%S')
df

,Dataset Name,Date
0,My First Dataset,2021-07-20 11:56:43
1,My Second Dataset,2021-07-20 14:35:18
2,My Third Dataset,2021-07-20 14:38:58


In [133]:
df.sort_values(by=['Date'],inplace=True,ascending=False,ignore_index=True)
df

,Dataset Name,Date
0,My Third Dataset,2021-07-20 14:38:58
1,My Second Dataset,2021-07-20 14:35:18
2,My First Dataset,2021-07-20 11:56:43


In [103]:
from copy import copy,deepcopy
query_dataset_SQL="""SELECT
    id,
    name,
    dataset_size,
    updated_at
FROM
    public.dataset;"""

dataset_list=db_fetchall(query_dataset_SQL,conn)
display(dataset_list)

# updated_at=[]
# dataset_name=[]
# for dataset in dataset_list:
#     without_timezone=dataset[3].strftime('%Y-%m-%d %H:%M:%S')
#     updated_at.append(without_timezone)
#     dataset_name.append(dataset[1])
datasets=list(dataset_list)
dataset_list=deepcopy(datasets)
datasets=[]
for dataset in dataset_list:
    dataset=list(dataset)

    # if dataset[2]:
        # dataset[2]=int(dataset[2])
    dataset[3]=dataset[3].strftime('%Y-%m-%d %H:%M:%S')
    datasets.append(dataset)
    
datasets
    





[(2,
  'My Second Dataset',
  4,
  datetime.datetime(2021, 7, 20, 14, 35, 18, 643911, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=480, name=None))),
 (3,
  'My Third Dataset',
  4,
  datetime.datetime(2021, 7, 20, 14, 38, 58, 453171, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=480, name=None)))]

[[2, 'My Second Dataset', 4, '2021-07-20 14:35:18'],
 [3, 'My Third Dataset', 4, '2021-07-20 14:38:58']]

In [117]:
df = pd.DataFrame( datasets,columns=['Dataset ID','Name','Dataset Size','Date/Time'])
# df=df.set_index('ID')
# df['Dataset Size']=df['Dataset Size'].astype('int32')
df.index.name=('No.')
df.dtypes
df.info()



df['Date/Time']=pd.to_datetime(df['Date/Time'],format='%Y-%m-%d %H:%M:%S')
df2=df.sort_values(by=['Date/Time'],ascending=False,ignore_index=True)
df2.index.name=('No.')
dfStyler = df2.style.set_properties(**{'text-align': 'center'}).set_table_styles([dict(selector='th', props=[('text-align', 'center')])])
dfStyler





<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Dataset ID    2 non-null      int64 
 1   Name          2 non-null      object
 2   Dataset Size  2 non-null      int64 
 3   Date/Time     2 non-null      object
dtypes: int64(2), object(2)
memory usage: 192.0+ bytes


,Dataset ID,Name,Dataset Size,Date/Time
No.,,,,
0,3,My Third Dataset,4,2021-07-20 14:38:58
1,2,My Second Dataset,4,2021-07-20 14:35:18


In [69]:
df2 = pd.DataFrame({'month': [1, 4, 7, 10],
                   'year': [2012, 2014, 2013, 2014]})

In [ ]:
df2=df2.set_index('year')


In [72]:
display(df2)

,month
year,
2012,1
2014,4
2013,7
2014,10


In [122]:
from queue import Queue
q=Queue()
for i in range(30):
    q.put(i)

print(q)
q.get_nowait()

0

In [129]:
db_fetchall("select name from public.dataset;",conn)

[('My Second Dataset',), ('My Third Dataset',)]

In [130]:
dataset_name=[]
for dataset in datasets:
    dataset_name.append(dataset[1])

dataset_name

['My Second Dataset', 'My Third Dataset']